# Ejercicio Módulo 2
**Inteligencia Artificial - CEIA - FIUBA**

**Damián Nicolás Smilovich**

En este ejercicio deben implementar un algoritmo de búsqueda que no sea **Búsqueda Primero en Anchura (BFS)** para resolver el problema de la Torre de Hanoi. La nota máxima dependerá del algoritmo implementado:

- **Búsqueda Primero en Profundidad**: nota máxima 6.
- **Búsqueda de Costo Uniforme**: nota máxima 6.
- **Búsqueda de Profundidad Limitada con Profundidad Iterativa**: nota máxima 7.
- **Búsqueda Voraz usando la heurística dada en el aula virtual**: nota máxima 8.
- **Búsqueda Voraz usando una heurística desarrollada por vos**: nota máxima 9.
- **Búsqueda A\* usando la heurística dada en el aula virtual**: nota máxima 9.
- **Búsqueda A\* usando una heurística desarrollada por vos**: nota máxima 10.

La función debe devolver la salida correspondiente a la solución encontrada o `None si no se encontró una solución.

Además, debe calcular métricas de rendimiento que, como mínimo, incluyan:

- `solution_found`: `True` si se encontró la solución, `False` en caso contrario.
- `nodes_explored`: cantidad de nodos explorados (entero).
- `states_visited`: cantidad de estados distintos visitados (entero).
- `nodes_in_frontier`: cantidad de nodos que quedaron en la frontera al finalizar la ejecución (entero).
- `max_depth`: máxima profundidad explorada (entero).
- `cost_total`: costo total para encontrar la solución (float).

In [ ]:
from aima_libs.hanoi_states import ProblemHanoi, StatesHanoi
from aima_libs.tree_hanoi import NodeHanoi

In [ ]:
def search_algorithm(number_disks=5) -> (NodeHanoi, dict):

    list_disks = [i for i in range(5, 0, -1)]
    initial_state = StatesHanoi(list_disks, [], [], max_disks=number_disks)
    goal_state = StatesHanoi([], [], list_disks, max_disks=number_disks)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)

    root = NodeHanoi(state=initial_state)
    nodes = [root]
    # sin esto entra en un loop infinito
    states_seen = set()

    max_depth = 0

    while True:
        if (len(nodes) == 0):
            metrics = {
                "solution_found": False,
                "nodes_explored": len(states_seen),
                "states_visited": len(states_seen),
                "nodes_in_frontier": len(nodes),
                "max_depth": max_depth,
                "cost_total": None,
            }
            return None, metrics

        # elimina el último nodo de la lista
        last_node = nodes.pop()
        max_depth = last_node.depth if last_node.depth > max_depth else max_depth
        # evita repetir estados, no se visitan nodos que se correspondan a estados ya visitados por lo que nodes_explored == states_visited
        states_seen.add(last_node.state)
        # chequea si es la solución
        if problem.goal_test(last_node.state):
            print(last_node.state)
            metrics = {
                "solution_found": True,
                "nodes_explored": len(states_seen),
                "states_visited": len(states_seen),
                "nodes_in_frontier": len(nodes) + 1, # +1 porque el nodo a chequear ya fue eliminado de nodes
                "max_depth": max_depth,
                "cost_total": last_node.state.accumulated_cost,
            }
            return last_node, metrics

        # expande el nodo
        frontier = last_node.expand(problem=problem)
        # y appendea los nodos frontera
        for node in frontier:
            # solo si no se corresponden con estados ya visitados
            if node.state not in states_seen:
                nodes.append(node)

Se prueba la función:

In [ ]:
solution, metrics = search_algorithm(number_disks=5)

Veamos las métricas:

In [ ]:
for key, value in metrics.items():
    print(f"{key}: {value}")

Veamos el camino de estados desde el principio a la solución:

In [ ]:
for nodos in solution.path():
    print(nodos)

Y las acciones que el agente debería aplicar para llegar al objetivo:

In [ ]:
for act in solution.solution():
    print(act)